In [1]:
# Libraries
import pandas as pd
import numpy as np
import scipy.stats as stats
import statsmodels.formula.api as smf
from statsmodels.stats.multitest import multipletests
from lifelines import KaplanMeierFitter
from lifelines.statistics import logrank_test
from lifelines import CoxPHFitter

# Load the dataset
data = pd.read_csv('DATA.csv')

data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 174 entries, 0 to 173
Data columns (total 19 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 174 non-null    object 
 1   age                174 non-null    int64  
 2   variant_histology  174 non-null    object 
 3   ypT_stage          174 non-null    object 
 4   ypN_stage          174 non-null    object 
 5   recurrence         173 non-null    object 
 6   fu_recurrence      174 non-null    float64
 7   dod                171 non-null    object 
 8   doc                170 non-null    object 
 9   fu_censor          174 non-null    float64
 10  sex                174 non-null    object 
 11  year_cystectomy    173 non-null    float64
 12  ypT_group          174 non-null    object 
 13  ypN_group          174 non-null    object 
 14  dre                174 non-null    object 
 15  ypT_group1         174 non-null    object 
 16  ypT_group2         174 non

In [2]:
# Convert variables to appropriate types
data['sex'] = data['sex'].astype('category')
data['variant_histology'] = data['variant_histology'].astype('category')
data['ypT_group'] = data['ypT_group'].astype('category')
data['ypN_group'] = data['ypN_group'].astype('category')

data['age'] = data['age'].astype('float')

binary_vars = ['recurrence', 'doc', 'dod', 'dre']
for var in binary_vars:
    data[var] = (data[var] == 'Yes').astype(int)

data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 174 entries, 0 to 173
Data columns (total 19 columns):
 #   Column             Non-Null Count  Dtype   
---  ------             --------------  -----   
 0   id                 174 non-null    object  
 1   age                174 non-null    float64 
 2   variant_histology  174 non-null    category
 3   ypT_stage          174 non-null    object  
 4   ypN_stage          174 non-null    object  
 5   recurrence         174 non-null    int64   
 6   fu_recurrence      174 non-null    float64 
 7   dod                174 non-null    int64   
 8   doc                174 non-null    int64   
 9   fu_censor          174 non-null    float64 
 10  sex                174 non-null    category
 11  year_cystectomy    173 non-null    float64 
 12  ypT_group          174 non-null    category
 13  ypN_group          174 non-null    category
 14  dre                174 non-null    int64   
 15  ypT_group1         174 non-null    object  
 16  ypT_grou

# Patient Demographics and Pre-Operative Characteristics

In [3]:
mean_age = data['age'].mean()
std_age = data['age'].std()
min_age = data['age'].min()
max_age = data['age'].max()
male_percent = data['sex'].value_counts(normalize=True).get('Male', 0) * 100
variant_histology_percent = data['variant_histology'].value_counts(normalize=True).get('Yes', 0) * 100

print(f"Average age: {mean_age:.1f} years (SD: {std_age:.1f}, range: {min_age}-{max_age} years)")
print(f"Percentage of male patients: {male_percent:.0f}%")
print(f"Percentage with variant histology: {variant_histology_percent:.0f}%")

Average age: 66.5 years (SD: 9.8, range: 24.0-86.0 years)
Percentage of male patients: 71%
Percentage with variant histology: 36%


# Pathological Response and Patient Stratification

In [4]:
study_group_count = data['ypT_group'].value_counts().get('pT0-pTa-pTis', 0)
control_group_count = data['ypT_group'].value_counts().get('pT2-pT3', 0)

print(f"STUDY GROUP: Number of patients with isolated lymph node metastases (ypT0/Tis/Ta): {study_group_count}")
print(f"CONTROL GROUP: Number of patients with persistent muscle-invasive disease (ypT2/3): {control_group_count}")

STUDY GROUP: Number of patients with isolated lymph node metastases (ypT0/Tis/Ta): 35
CONTROL GROUP: Number of patients with persistent muscle-invasive disease (ypT2/3): 139


# Impact of Pathological Response on Recurrence

In [5]:
recurrence_rate = data['recurrence'].value_counts(normalize=True).get(1, 0) * 100
print(f"Overall recurrence rate: {recurrence_rate:.0f}%")

Overall recurrence rate: 33%


In [6]:
# Unadjusted analysis
model_recurrence_unadj = smf.logit("recurrence ~ ypT_group", data=data).fit(disp=False)
or_recurrence_unadj = np.exp(model_recurrence_unadj.params['ypT_group[T.pT2-pT3]'])
ci_recurrence_unadj = np.exp(model_recurrence_unadj.conf_int().loc['ypT_group[T.pT2-pT3]'])
pval_recurrence_unadj = model_recurrence_unadj.pvalues['ypT_group[T.pT2-pT3]']

print(f"Unadjusted OR for recurrence (ypT2/3 vs ypT0/Ta/Tis): {or_recurrence_unadj:.2f}, 95% CI: [{ci_recurrence_unadj[0]:.2f}, {ci_recurrence_unadj[1]:.2f}], p={pval_recurrence_unadj:.3f}")

Unadjusted OR for recurrence (ypT2/3 vs ypT0/Ta/Tis): 0.43, 95% CI: [0.20, 0.91], p=0.028


In [7]:
# Adjusted analysis
model_recurrence_adj = smf.logit("recurrence ~ ypT_group + age + sex + variant_histology", data=data).fit(disp=False)
or_recurrence_adj = np.exp(model_recurrence_adj.params['ypT_group[T.pT2-pT3]'])
ci_recurrence_adj = np.exp(model_recurrence_adj.conf_int().loc['ypT_group[T.pT2-pT3]'])
pval_recurrence_adj = model_recurrence_adj.pvalues['ypT_group[T.pT2-pT3]']

print(f"Adjusted OR for recurrence (ypT2/3 vs ypT0/Ta/Tis): {or_recurrence_adj:.2f}, 95% CI: [{ci_recurrence_adj[0]:.2f}, {ci_recurrence_adj[1]:.2f}], p={pval_recurrence_adj:.3f}")

Adjusted OR for recurrence (ypT2/3 vs ypT0/Ta/Tis): 0.43, 95% CI: [0.20, 0.95], p=0.036


# Influence of Pathological Response and Tumor Characteristics on Mortality

In [8]:
# Analysis dod
dod_rate = data['dod'].value_counts(normalize=True).get(1, 0) * 100
print(f"Disease-specific mortality: {dod_rate:.0f}%")

Disease-specific mortality: 24%


In [9]:
# Unadjusted analysis dod
model_dod_unadj = smf.logit("dod ~ ypT_group", data=data).fit(disp=False)
or_dod_unadj = np.exp(model_dod_unadj.params['ypT_group[T.pT2-pT3]'])
ci_dod_unadj = np.exp(model_dod_unadj.conf_int().loc['ypT_group[T.pT2-pT3]'])
pval_dod_unadj = model_dod_unadj.pvalues['ypT_group[T.pT2-pT3]']

print(f"Unadjusted OR for disease-specific death (ypT2/3 vs ypT0/Ta/Tis): {or_dod_unadj:.2f}, 95% CI: [{ci_dod_unadj[0]:.2f}, {ci_dod_unadj[1]:.2f}], p={pval_dod_unadj:.3f}")

Unadjusted OR for disease-specific death (ypT2/3 vs ypT0/Ta/Tis): 0.72, 95% CI: [0.31, 1.65], p=0.436


In [10]:
# Adjusted analysis dod
model_dod_adj = smf.logit("dod ~ ypT_group + age + sex + variant_histology", data=data).fit(disp=False)
or_dod_adj = np.exp(model_dod_adj.params['ypT_group[T.pT2-pT3]'])
ci_dod_adj = np.exp(model_dod_adj.conf_int().loc['ypT_group[T.pT2-pT3]'])
pval_dod_adj = model_dod_adj.pvalues['ypT_group[T.pT2-pT3]']
print(f"Adjusted OR for disease-specific death (ypT2/3 vs ypT0/Ta/Tis): {or_dod_adj:.2f}, 95% CI: [{ci_dod_adj[0]:.2f}, {ci_dod_adj[1]:.2f}], p={pval_dod_adj:.3f}")

Adjusted OR for disease-specific death (ypT2/3 vs ypT0/Ta/Tis): 0.70, 95% CI: [0.29, 1.64], p=0.407


In [11]:
# Analysis dre
dre_rate = data['dre'].value_counts(normalize=True).get(1, 0) * 100
print(f"Disease-related event rate: {dre_rate:.0f}%")

Disease-related event rate: 41%


In [12]:
# Unadjusted analysis dre
model_dre_unadj = smf.logit("dre ~ ypT_group", data=data).fit(disp=False)
or_dre_unadj = np.exp(model_dre_unadj.params['ypT_group[T.pT2-pT3]'])
ci_dre_unadj = np.exp(model_dre_unadj.conf_int().loc['ypT_group[T.pT2-pT3]'])
pval_dre_unadj = model_dre_unadj.pvalues['ypT_group[T.pT2-pT3]']
print(f"Unadjusted OR for a disease-related event (ypT2/3 vs ypT0/Ta/Tis): {or_dre_unadj:.2f}, 95% CI: [{ci_dre_unadj[0]:.2f}, {ci_dre_unadj[1]:.2f}], p={pval_dre_unadj:.3f}")

Unadjusted OR for a disease-related event (ypT2/3 vs ypT0/Ta/Tis): 0.45, 95% CI: [0.21, 0.95], p=0.037


In [13]:
# Adjusted analysis dre
model_dre_adj = smf.logit("dre ~ ypT_group + age + sex + variant_histology", data=data).fit(disp=False)
or_dre_adj = np.exp(model_dre_adj.params['ypT_group[T.pT2-pT3]'])
ci_dre_adj = np.exp(model_dre_adj.conf_int().loc['ypT_group[T.pT2-pT3]'])
pval_dre_adj = model_dre_adj.pvalues['ypT_group[T.pT2-pT3]']
print(f"Adjusted OR for a disease-related event (ypT2/3 vs ypT0/Ta/Tis): {or_dre_adj:.2f}, 95% CI: [{ci_dre_adj[0]:.2f}, {ci_dre_adj[1]:.2f}], p={pval_dre_adj:.3f}")

Adjusted OR for a disease-related event (ypT2/3 vs ypT0/Ta/Tis): 0.46, 95% CI: [0.21, 0.99], p=0.048


# Prognostic Significance of Pathological Response and Variant Histology on Survival

In [14]:
# Overall survival analysis
kmf_os = KaplanMeierFitter()
kmf_os.fit(data['fu_censor'], event_observed=data['dod'].apply(lambda x: 1 if x==1 else 0), label="Overall Survival")
logrank_os = logrank_test(data['fu_censor'][data['ypT_group'] == 'pT0-pTa-pTis'],
                          data['fu_censor'][data['ypT_group'] == 'pT2-pT3'],
                          event_observed_A=data['dod'][data['ypT_group'] == 'pT0-pTa-pTis'].apply(lambda x: 1 if x=='Yes' else 0),
                          event_observed_B=data['dod'][data['ypT_group'] == 'pT2-pT3'].apply(lambda x: 1 if x=='Yes' else 0))
print(f"Log-rank test for overall survival: p = {logrank_os.p_value:.3f}")

Log-rank test for overall survival: p = 1.000


In [15]:

# Univariate model for ypT group
cph_os_univar = CoxPHFitter()
cph_os_univar.fit(data, 'fu_censor', event_col='dod', formula='ypT_group')
hr_os_univar_ypT = cph_os_univar.hazard_ratios_['ypT_group[T.pT2-pT3]']
ci_os_univar_ypT = cph_os_univar.confidence_intervals_.loc['ypT_group[T.pT2-pT3]']
pval_os_univar_ypT = cph_os_univar.summary.loc['ypT_group[T.pT2-pT3]', 'p']

print(f"Univariate HR for overall survival (ypT2/3 vs ypT0/Ta/Tis): {hr_os_univar_ypT:.2f}, 95% CI: [{ci_os_univar_ypT.iloc[0]:.2f}, {ci_os_univar_ypT.iloc[1]:.2f}], p={pval_os_univar_ypT:.3f}")

Univariate HR for overall survival (ypT2/3 vs ypT0/Ta/Tis): 0.93, 95% CI: [-0.78, 0.64], p=0.847


In [16]:
# Multivariate model
cph_os_multi = CoxPHFitter()
cph_os_multi.fit(data, 'fu_censor', event_col='dod', formula='ypT_group + age + sex + variant_histology')
hr_os_multi = cph_os_multi.hazard_ratios_['ypT_group[T.pT2-pT3]']
ci_os_multi = cph_os_multi.confidence_intervals_.loc['ypT_group[T.pT2-pT3]']
pval_os_multi = cph_os_multi.summary.loc['ypT_group[T.pT2-pT3]','p']

print(f"Adjusted HR for overall survival (ypT2/3 vs ypT0/Ta/Tis): {hr_os_multi:.2f}, 95% CI: [{ci_os_multi.iloc[0]:.2f}, {ci_os_multi.iloc[1]:.2f}], p={pval_os_multi:.3f}")

Adjusted HR for overall survival (ypT2/3 vs ypT0/Ta/Tis): 0.91, 95% CI: [-0.83, 0.64], p=0.799


In [17]:
# Disease-specific survival analysis
kmf_dss = KaplanMeierFitter()
kmf_dss.fit(data['fu_censor'], event_observed=data['dod'].apply(lambda x: 1 if x==1 else 0), label="Disease-specific Survival")
logrank_dss = logrank_test(data['fu_censor'][data['ypT_group'] == 'pT0-pTa-pTis'],
                         data['fu_censor'][data['ypT_group'] == 'pT2-pT3'],
                          event_observed_A=data['dod'][data['ypT_group'] == 'pT0-pTa-pTis'].apply(lambda x: 1 if x=='Yes' else 0),
                          event_observed_B=data['dod'][data['ypT_group'] == 'pT2-pT3'].apply(lambda x: 1 if x=='Yes' else 0))

print(f"Log-rank test for disease-specific survival: p = {logrank_dss.p_value:.3f}")

Log-rank test for disease-specific survival: p = 1.000


In [18]:
# Univariate DSS
cph_dss_univar = CoxPHFitter()
cph_dss_univar.fit(data, 'fu_censor', event_col='dod', formula='ypT_group')
hr_dss_univar_ypT = cph_dss_univar.hazard_ratios_['ypT_group[T.pT2-pT3]']
ci_dss_univar_ypT = cph_dss_univar.confidence_intervals_.loc['ypT_group[T.pT2-pT3]']
pval_dss_univar_ypT = cph_dss_univar.summary.loc['ypT_group[T.pT2-pT3]','p']

print(f"Univariate HR for disease-specific survival (ypT2/3 vs ypT0/Ta/Tis): {hr_dss_univar_ypT:.2f}, 95% CI: [{ci_dss_univar_ypT.iloc[0]:.2f}, {ci_dss_univar_ypT.iloc[1]:.2f}], p={pval_dss_univar_ypT:.3f}")

Univariate HR for disease-specific survival (ypT2/3 vs ypT0/Ta/Tis): 0.93, 95% CI: [-0.78, 0.64], p=0.847


In [19]:
cph_dss_multi = CoxPHFitter()
cph_dss_multi.fit(data, 'fu_censor', event_col='dod', formula='ypT_group + age + sex + variant_histology')
hr_dss_multi = cph_dss_multi.hazard_ratios_['ypT_group[T.pT2-pT3]']
ci_dss_multi = cph_dss_multi.confidence_intervals_.loc['ypT_group[T.pT2-pT3]']
pval_dss_multi = cph_dss_multi.summary.loc['ypT_group[T.pT2-pT3]','p']

print(f"Adjusted HR for disease-specific survival (ypT2/3 vs ypT0/Ta/Tis): {hr_dss_multi:.2f}, 95% CI: [{ci_dss_multi.iloc[0]:.2f}, {ci_dss_multi.iloc[1]:.2f}], p={pval_dss_multi:.3f}")

Adjusted HR for disease-specific survival (ypT2/3 vs ypT0/Ta/Tis): 0.91, 95% CI: [-0.83, 0.64], p=0.799
